In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import copy
import pandas as pd
import time
import re
import sys
from lib.network_mapper import network_mapper_function
from lib.NR3_OpenDSSvec_splitup import NR3_function

In [2]:
fp = "C:\\Users\\kathl\\Desktop\\LinDist3Flow\\20180601\\PYTHON\\"
fn = 'compare_lindist3flow_05node_threephase_unbalanced_oscillation_03.txt'
network1 = network_mapper_function(fp, fn)

nnode1 = network1.nodes.nnode
nline1 = network1.lines.nline

# Network paramaters

nnode = network1.nodes.nnode
nline = network1.lines.nline

# Load parameters

network1.loads.aPQ = 0.85*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI =0*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.15*np.ones((3,nnode1))*(network1.loads.spu != 0)


network1.loads.spu = 1*network1.loads.spu
network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))


fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'

BASE
Vbase: 2771.2812921102036
Sbase: 1000000.0
Ibase: 360.8439182435161
Zbase: 7.68

NODES
nnode: 7
nodelist: ['sub', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06']
phases: ['abc', 'abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 0]]

LINES
nline: 6
TXnode: ['sub', 'A01', 'A02', 'A03', 'A03', 'A05']
RXnode: ['A01', 'A02', 'A03', 'A04', 'A05', 'A06']
TXnum: [0 1 2 3 3 5]
RXnum: [1 2 3 4 5 6]
phases: ['abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 0]
 [1 1 1 0 1 1]
 [1 1 1 1 0 0]]
config: ['sub_a01', 'a01_a02', 'a02_a03', 'a03_a04', 'a03_a05', 'a05_a06']
length: [1. 1. 1. 1. 1. 1.]

NODES + LINES
inlines:
 [[-1  0  1  2  3  4  5]]
innodes:
 [[-1  0  1  2  3  3  5]]
outlines:
 [[ 0  1  2  3 -1  5 -1]
 [-1 -1 -1  4 -1 -1 -1]]
outnodes:
 [[ 1  2  3  4 -1  6 -1]
 [-1 -1 -1  5 -1 -1 -1]]

CONFIGS
nconf: 6
conflist: ['sub_a01', 'a01_a02', 'a02_a03', 'a03_a04', 'a03_a05', 'a05_a06']
sub_a01 - FZpl:
 [[0.00195312+0.00130208j 0.00065104+0.00520833j 0.

In [3]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(network1, fn, slackidx,Vslack,None,None,None)

DSSException: (99, 'Error 906 Reported From OpenDSS Intrinsic Function: \r\nMakeNewCircuit\r\n\r\nError Description: \r\nCannot create new circuit.\r\n\r\nProbable Cause: \r\nMax. Circuits Exceeded.\r\n(Max no. of circuits=1)')

---

In [ ]:
dss.run_command('Redirect ' + fn)
dss.Solution.Solve()
for i in range(len(dss.Circuit.AllBusNames())):
    #print(dss.Circuit.AllBusNames()[i])
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[i])
    print(dss.CktElement.Powers())
for j in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[j])
    print(dss.CktElement.Powers())
    print(len(dss.CktElement.Powers()))
    print("\n")

In [ ]:
def get_power_based_on_bus(busname, cplx):
    
    dss.Circuit.SetActiveBus(busname)
   
    for j in range(len(dss.Loads.AllNames())):
        if dss.Circuit.AllBusNames()[n] in dss.Loads.AllNames()[j]:
            dss.Loads.Name(dss.Loads.AllNames()[j])
            if cplx == 0:
                return dss.Loads.kW()
            else:
                return dss.Loads.kvar()
        

        #print('hey')
get_power_based_on_bus(dss.Circuit.AllBusNames()[6], 1)

In [ ]:
print(nnode)